Import packages

In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline

import numpy as np
import pandas as pd
import dask.dataframe as dd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
color = sns.color_palette()

# special matplotlib argument for improved plots
from matplotlib import rcParams

pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns", 999)

C:\Users\cepps\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Read in FBXSITA final mailfile

In [2]:
mb_pbfbxs_ita_0518 = pd.read_csv('E:/Pallavi/2018/rev1/rev2/mailfile/mailfiles/561) 117487A_RMC_PBFBXS5.0_18ITA_Final_Mailfile.TXT', 
                                 sep='\t', na_values='.', encoding="ISO-8859-1", error_bad_lines=False, low_memory=False, 
                                parse_dates = ["DOB", "Drop_Date", "Closed_Date", "POffDate"])
mb_pbfbxs_ita_0518.columns = mb_pbfbxs_ita_0518.columns.str.strip().str.lower().str.replace(' ', '_') 

In [3]:
mb_pbfbxs_ita_0518['zip5'] = mb_pbfbxs_ita_0518['zip5'].astype(int)
mb_pbfbxs_ita_0518['zip5'] = mb_pbfbxs_ita_0518['zip5'].astype(str)
mb_pbfbxs_ita_0518['zip4'] = mb_pbfbxs_ita_0518['zip4'].astype(int)
mb_pbfbxs_ita_0518['zip4'] = mb_pbfbxs_ita_0518['zip4'].astype(str)
mb_pbfbxs_ita_0518['qseqnum'] = mb_pbfbxs_ita_0518['qseqnum'].astype(int)
mb_pbfbxs_ita_0518['qseqnum'] = mb_pbfbxs_ita_0518['qseqnum'].astype(str)
mb_pbfbxs_ita_0518['branchnumber'] = mb_pbfbxs_ita_0518['branchnumber'].astype(int)
mb_pbfbxs_ita_0518['branchnumber'] = mb_pbfbxs_ita_0518['branchnumber'].astype(str)
mb_pbfbxs_ita_0518['distance_to_branch'] = pd.to_numeric(mb_pbfbxs_ita_0518['distance_to_branch'], errors='coerce')
mb_pbfbxs_ita_0518['amt_given1'] = mb_pbfbxs_ita_0518['amt_given1'].str.replace(',', '')
mb_pbfbxs_ita_0518['amt_given1'] = mb_pbfbxs_ita_0518['amt_given1'].str.replace('$', '')
mb_pbfbxs_ita_0518['amt_given1'] = pd.to_numeric(mb_pbfbxs_ita_0518['amt_given1'], errors='coerce')
mb_pbfbxs_ita_0518['up_to_offer'] = mb_pbfbxs_ita_0518['up_to_offer'].str.replace(',', '')
mb_pbfbxs_ita_0518['up_to_offer'] = mb_pbfbxs_ita_0518['up_to_offer'].str.replace('$', '')
mb_pbfbxs_ita_0518['up_to_offer'] = pd.to_numeric(mb_pbfbxs_ita_0518['up_to_offer'], errors='coerce')
mb_pbfbxs_ita_0518['from_offer_amount'] = pd.to_numeric(mb_pbfbxs_ita_0518['from_offer_amount'], errors='coerce')
mb_pbfbxs_ita_0518['fico'] = pd.to_numeric(mb_pbfbxs_ita_0518['fico'], errors='coerce')

Print dimentions

In [4]:
mb_pbfbxs_ita_0518.shape

(97500, 49)

In [5]:
mb_pbfbxs_ita_0518.head()

,campaign_id,hhfilecode,batch,lettercd,ss7,name1,addr2,street,city,state,zip5,zip4,dob,made_unmade,qseqnum,drop_date,closed_date,mla_status,branchnumber,branchcompany,branchstreetaddress,branchcity,branchstate,branchzip,branchphone,branchfax,branchmanager,branchsupervisor,branchstatevp,distance_to_branch,conprofile,bracctno,cifno,mgc,month_split,fname,mname,lname,amt_given1,up_to_offer,from_offer_amount,fico_range_25pt,state1,test_code,fico,poffdate,rmc_phone,rmc_cellphone,unnamed:_48
0,FBITA5.0_2018,FB,FMITA,FB,4690645,Christine Pennington,,208 Canvasback Dr.,Bloomingdale,GA,31302,8031,19881023,,1,2018-05-07,2018-06-05,N,1101,Regional Finance,"6409 Abercorn Street, Suite A",Savannah,GA,31405-5796,(912) 354-1144,(912) 328-5408,Debbie Sanders,Mike Votava,Heather Meares,14.808367,,1101003466,B1571550,,,CHRISTINE,,PENNINGTON,NaN,7000,600.0,575-599,GA,,593.0,2017-09-19,(912)547-2071,(900)000-0000,NaN
1,FBITA5.0_2018,FB,FMITA,FB,4530510,Tina Rustin,,5 Douglas Ct.,Bloomingdale,GA,31302,9218,19701105,,2,2018-05-07,2018-06-05,N,1101,Regional Finance,"6409 Abercorn Street, Suite A",Savannah,GA,31405-5796,(912) 354-1144,(912) 328-5408,Debbie Sanders,Mike Votava,Heather Meares,13.180202,,1101102475,B1406590,,,TINA,,RUSTIN,NaN,7000,600.0,,GA,,NaN,2016-03-01,(912)856-4204,,NaN
2,FBITA5.0_2018,FB,FMITA,FB,8495068,Kenneth Pritsch,,109 Sandstone Dr.,Bloomingdale,GA,31302,8111,19620726,,3,2018-05-07,2018-06-05,N,1101,Regional Finance,"6409 Abercorn Street, Suite A",Savannah,GA,31405-5796,(912) 354-1144,(912) 328-5408,Debbie Sanders,Mike Votava,Heather Meares,18.732519,,1101005213,B1623331,,,KENNETH,A,PRITSCH,NaN,7000,600.0,625-649,GA,,639.0,2017-12-01,(912)330-0626,(218)994-9155,NaN
3,FBITA5.0_2018,FB,FMITA,FB,7432608,Angela Parker,,1024 Jones Way SE,Darien,GA,31305,5800,19811202,,4,2018-05-07,2018-06-05,N,1102,Regional Finance,3421-6 Cypress Mill Road,Brunswick,GA,31520-2876,(912) 261-2882,(912) 231-5079,Becky Cross,Mike Votava,Heather Meares,17.899126,,1102002495,B1650604,,,ANGELA,G,PARKER,NaN,7000,600.0,600-624,GA,,608.0,2017-05-24,(912)617-1510,,NaN
4,FBITA5.0_2018,FB,FMITA,FB,8354805,Vermiel Pierce Jr.,,1028 Waters Ct. SE,Darien,GA,31305,5828,19751023,,5,2018-05-07,2018-06-05,N,1102,Regional Finance,3421-6 Cypress Mill Road,Brunswick,GA,31520-2876,(912) 261-2882,(912) 231-5079,Becky Cross,Mike Votava,Heather Meares,17.930010,999999999999,1102003613,B1629385,,,VERMIEL,A,PIERCE JR,NaN,7000,600.0,625-649,GA,,626.0,2018-02-13,(912)437-2042,,NaN


Print summary of variables

In [6]:
mb_pbfbxs_ita_0518.describe(include = 'all')

,campaign_id,hhfilecode,batch,lettercd,ss7,name1,addr2,street,city,state,zip5,zip4,dob,made_unmade,qseqnum,drop_date,closed_date,mla_status,branchnumber,branchcompany,branchstreetaddress,branchcity,branchstate,branchzip,branchphone,branchfax,branchmanager,branchsupervisor,branchstatevp,distance_to_branch,conprofile,bracctno,cifno,mgc,month_split,fname,mname,lname,amt_given1,up_to_offer,from_offer_amount,fico_range_25pt,state1,test_code,fico,poffdate,rmc_phone,rmc_cellphone,unnamed:_48
count,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500.000000,97500,97500,97500,97500,97500,97500,97499,97496,0.0,97500.000000,97365.000000,97500,97500,97500,97280.000000,97500,97500,97500,0.0
unique,3,2,2,2,97231,89107,2098,94465,2661,9,4285,9713,20688,1,97500,1,1,2,340,1,340,260,9,340,340,340,337,40,6,NaN,60,97500,88174,1,1,15100,416,24862,NaN,NaN,NaN,20,9,1,NaN,1112,57498,15311,NaN
top,FBITA5.0_2018,FB,FMITA,FB,3709069,Maria Garcia,,P.O. Box 93,Houston,TX,79936,3614,,,12833,2018-05-07 00:00:00,2018-06-05 00:00:00,N,557,Regional Finance,1703 Shaver Street,San Antonio,TX,77502-2027,(832) 786-3844,(832) 559-0679,Crystal Martinez,Sandra Lopez,Michael Johnson,NaN,999999999999,TCI573429,,,,MARIA,,SMITH,NaN,NaN,NaN,625-649,,,NaN,,,()-,NaN
freq,88173,88173,49118,88173,3,37,94019,7,4900,39323,485,38,14693,97500,1,97500,97500,82800,756,97500,756,4339,39323,756,756,756,756,5015,35093,NaN,53603,1,9327,97500,97500,1289,91706,789,NaN,NaN,NaN,21416,39340,97500,NaN,9327,39256,64334,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-07 00:00:00,2018-06-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-07 00:00:00,2018-06-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.153210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6874.492308,609.512659,NaN,NaN,NaN,632.389967,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.956715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331.295393,29.339121,NaN,NaN,NaN,74.681539,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000.000000,600.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.777503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7000.000000,600.000000,NaN,NaN,NaN,597.000000,NaN,NaN,NaN,NaN


Find unique campaign IDs

Set memory usage parameters to `deep` for accurate measure of memory usage

In [7]:
mb_pbfbxs_ita_0518.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97500 entries, 0 to 97499
Data columns (total 49 columns):
campaign_id            97500 non-null object
hhfilecode             97500 non-null object
batch                  97500 non-null object
lettercd               97500 non-null object
ss7                    97500 non-null object
name1                  97500 non-null object
addr2                  97500 non-null object
street                 97500 non-null object
city                   97500 non-null object
state                  97500 non-null object
zip5                   97500 non-null object
zip4                   97500 non-null object
dob                    97500 non-null object
made_unmade            97500 non-null object
qseqnum                97500 non-null object
drop_date              97500 non-null datetime64[ns]
closed_date            97500 non-null datetime64[ns]
mla_status             97500 non-null object
branchnumber           97500 non-null object
branchcompany       

Average memory usage per data type

In [8]:
for dtype in ['float', 'int', 'object']:
    selected_dtype = mb_pbfbxs_ita_0518.select_dtypes(include = [dtype])
    mean_usage_b = selected_dtype.memory_usage(deep = True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 0.62 MB
Average memory usage for int columns: 0.00 MB
Average memory usage for object columns: 5.84 MB


In [9]:
import numpy as np
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



In [10]:
# We're going to be calculating memory usage a lot,
# so we'll create a function to save us some time!

def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep = True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep = True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

mb_pbfbxs_ita_0518_int = mb_pbfbxs_ita_0518.select_dtypes(include = ['int'])
converted_int = mb_pbfbxs_ita_0518_int.apply(pd.to_numeric, downcast = 'unsigned')

print(mem_usage(mb_pbfbxs_ita_0518_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([mb_pbfbxs_ita_0518_int.dtypes, converted_int.dtypes], axis = 1)
compare_ints.columns = ['before', 'after']
compare_ints.apply(pd.Series.value_counts)

0.00 MB
0.00 MB


,before,after


In [11]:
mb_pbfbxs_ita_0518_float = mb_pbfbxs_ita_0518.select_dtypes(include=['float'])
converted_float = mb_pbfbxs_ita_0518_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(mb_pbfbxs_ita_0518_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([mb_pbfbxs_ita_0518_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

3.72 MB
1.86 MB


,before,after
float32,NaN,5.0
float64,5.0,NaN


In [12]:
optimized_mb_pbfbxs_ita_0518 = mb_pbfbxs_ita_0518.copy()

optimized_mb_pbfbxs_ita_0518[converted_int.columns] = converted_int
optimized_mb_pbfbxs_ita_0518[converted_float.columns] = converted_float

print(mem_usage(mb_pbfbxs_ita_0518))
print(mem_usage(optimized_mb_pbfbxs_ita_0518))

251.24 MB
249.38 MB


In [13]:
from sys import getsizeof

s1 = 'working out'
s2 = 'memory usage for'
s3 = 'strings in python is fun!'
s4 = 'strings in python is fun!'

for s in [s1, s2, s3, s4]:
    print(getsizeof(s))

60
65
74
74


In [14]:
obj_series = pd.Series(['working out',
                          'memory usage for',
                          'strings in python is fun!',
                          'strings in python is fun!'])
obj_series.apply(getsizeof)

0    60
1    65
2    74
3    74
dtype: int64

In [15]:
mb_pbfbxs_ita_0518_obj = mb_pbfbxs_ita_0518.select_dtypes(include=['object']).copy()
mb_pbfbxs_ita_0518_obj.describe()

,campaign_id,hhfilecode,batch,lettercd,ss7,name1,addr2,street,city,state,zip5,zip4,dob,made_unmade,qseqnum,mla_status,branchnumber,branchcompany,branchstreetaddress,branchcity,branchstate,branchzip,branchphone,branchfax,branchmanager,branchsupervisor,branchstatevp,conprofile,bracctno,cifno,mgc,month_split,fname,mname,lname,fico_range_25pt,state1,test_code,poffdate,rmc_phone,rmc_cellphone
count,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97500,97499,97496,97500,97500,97500,97500,97500,97500
unique,3,2,2,2,97231,89107,2098,94465,2661,9,4285,9713,20688,1,97500,2,340,1,340,260,9,340,340,340,337,40,6,60,97500,88174,1,1,15100,416,24862,20,9,1,1112,57498,15311
top,FBITA5.0_2018,FB,FMITA,FB,3709069,Maria Garcia,,P.O. Box 93,Houston,TX,79936,3614,,,12833,N,557,Regional Finance,1703 Shaver Street,San Antonio,TX,77502-2027,(832) 786-3844,(832) 559-0679,Crystal Martinez,Sandra Lopez,Michael Johnson,999999999999,TCI573429,,,,MARIA,,SMITH,625-649,,,,,()-
freq,88173,88173,49118,88173,3,37,94019,7,4900,39323,485,38,14693,97500,1,82800,756,97500,756,4339,39323,756,756,756,756,5015,35093,53603,1,9327,97500,97500,1289,91706,789,21416,39340,97500,9327,39256,64334


In [16]:
campaign_id = mb_pbfbxs_ita_0518_obj.campaign_id
print(campaign_id.head())

campaign_id_cat = campaign_id.astype('category')
print(campaign_id_cat.head())

0    FBITA5.0_2018
1    FBITA5.0_2018
2    FBITA5.0_2018
3    FBITA5.0_2018
4    FBITA5.0_2018
Name: campaign_id, dtype: object
0    FBITA5.0_2018
1    FBITA5.0_2018
2    FBITA5.0_2018
3    FBITA5.0_2018
4    FBITA5.0_2018
Name: campaign_id, dtype: category
Categories (3, object): [AutoXSITA5.0_2018, FBITA5.0_2018, RetailXSITA5.0_2018]


In [17]:
campaign_id_cat.head().cat.codes

0    1
1    1
2    1
3    1
4    1
dtype: int8

In [18]:
print(mem_usage(campaign_id))
print(mem_usage(campaign_id_cat))

6.56 MB
0.09 MB


In [19]:
converted_obj = pd.DataFrame()

for col in mb_pbfbxs_ita_0518_obj.columns:
    num_unique_values = len(mb_pbfbxs_ita_0518_obj[col].unique())
    num_total_values = len(mb_pbfbxs_ita_0518_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = mb_pbfbxs_ita_0518_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = mb_pbfbxs_ita_0518_obj[col]

In [20]:
print(mem_usage(mb_pbfbxs_ita_0518_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([mb_pbfbxs_ita_0518_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

245.29 MB
57.47 MB


,before,after
object,41.0,7
category,NaN,4
category,NaN,2
category,NaN,2
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1


In [21]:
optimized_mb_pbfbxs_ita_0518[converted_obj.columns] = converted_obj

mem_usage(optimized_mb_pbfbxs_ita_0518)

'61.56 MB'

In [22]:
drop_date = optimized_mb_pbfbxs_ita_0518.drop_date
print(mem_usage(drop_date))
drop_date.head()

0.74 MB


0   2018-05-07
1   2018-05-07
2   2018-05-07
3   2018-05-07
4   2018-05-07
Name: drop_date, dtype: datetime64[ns]

In [23]:
optimized_mb_pbfbxs_ita_0518['drop_date'] = pd.to_datetime(drop_date,format='%Y%m%d')

print(mem_usage(optimized_mb_pbfbxs_ita_0518))
optimized_mb_pbfbxs_ita_0518.drop_date.head()

61.56 MB


0   2018-05-07
1   2018-05-07
2   2018-05-07
3   2018-05-07
4   2018-05-07
Name: drop_date, dtype: datetime64[ns]

In [24]:
closed_date = optimized_mb_pbfbxs_ita_0518.closed_date
print(mem_usage(closed_date))
closed_date.head()

0.74 MB


0   2018-06-05
1   2018-06-05
2   2018-06-05
3   2018-06-05
4   2018-06-05
Name: closed_date, dtype: datetime64[ns]

In [25]:
optimized_mb_pbfbxs_ita_0518['closed_date'] = pd.to_datetime(closed_date,format='%Y%m%d')

print(mem_usage(optimized_mb_pbfbxs_ita_0518))
optimized_mb_pbfbxs_ita_0518.closed_date.head()

61.56 MB


0   2018-06-05
1   2018-06-05
2   2018-06-05
3   2018-06-05
4   2018-06-05
Name: closed_date, dtype: datetime64[ns]

In [26]:
dob = optimized_mb_pbfbxs_ita_0518.dob
print(mem_usage(dob))
dob.head()

2.09 MB


0    19881023
1    19701105
2    19620726
3    19811202
4    19751023
Name: dob, dtype: category
Categories (20688, object): [, 18810310, 19000101, 19000104, ..., 19990605, 20010924, 20161013, 20211114]

In [27]:
optimized_mb_pbfbxs_ita_0518['dob'] = pd.to_datetime(dob,format='%Y%m%d', errors = 'coerce')

print(mem_usage(optimized_mb_pbfbxs_ita_0518))
optimized_mb_pbfbxs_ita_0518.dob.head()

60.21 MB


0   1988-10-23
1   1970-11-05
2   1962-07-26
3   1981-12-02
4   1975-10-23
Name: dob, dtype: datetime64[ns]

In [28]:
poffdate = optimized_mb_pbfbxs_ita_0518.poffdate
print(mem_usage(poffdate))
poffdate.head()

0.30 MB


0    2017-09-19
1    2016-03-01
2    2017-12-01
3    2017-05-24
4    2018-02-13
Name: poffdate, dtype: category
Categories (1112, object): [, 2014-11-17, 2014-11-18, 2014-11-19, ..., 2018-03-17, 2018-03-18, 2018-03-30, 2018-04-02]

In [29]:
optimized_mb_pbfbxs_ita_0518['poffdate'] = pd.to_datetime(poffdate,format='%Y%m%d', errors = 'coerce')

print(mem_usage(optimized_mb_pbfbxs_ita_0518))
optimized_mb_pbfbxs_ita_0518.poffdate.head()

60.66 MB


0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: poffdate, dtype: datetime64[ns]

In [30]:
optimized_mb_pbfbxs_ita_0518.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97500 entries, 0 to 97499
Data columns (total 49 columns):
campaign_id            97500 non-null category
hhfilecode             97500 non-null category
batch                  97500 non-null category
lettercd               97500 non-null category
ss7                    97500 non-null object
name1                  97500 non-null object
addr2                  97500 non-null category
street                 97500 non-null object
city                   97500 non-null category
state                  97500 non-null category
zip5                   97500 non-null category
zip4                   97500 non-null category
dob                    82807 non-null datetime64[ns]
made_unmade            97500 non-null category
qseqnum                97500 non-null object
drop_date              97500 non-null datetime64[ns]
closed_date            97500 non-null datetime64[ns]
mla_status             97500 non-null category
branchnumber           97500 non-nu

In [31]:
dtypes = optimized_mb_pbfbxs_ita_0518.drop(['drop_date', 'closed_date', 'dob', 'poffdate'],axis=1).dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint

preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'addr2': 'category',
    'batch': 'category',
    'campaign_id': 'category',
    'city': 'category',
    'hhfilecode': 'category',
    'lettercd': 'category',
    'name1': 'object',
    'ss7': 'object',
    'state': 'category',
    'street': 'object'}


In [32]:
optimized_mb_pbfbxs_ita_0518.to_csv(r'C:\Users\cepps\RMC\Mail Files\Optimized_Mail_Files\fbxs_ita\optimized_mb_pbfbxs_ita_0518.txt', 
                                    sep = '\t', mode='a')

In [33]:
optimized_mb_pbfbxs_ita_0518 = pd.read_csv('C:/Users/cepps/RMC/Mail Files/Optimized_Mail_Files/fbxs_ita/optimized_mb_pbfbxs_ita_0518.txt', 
                                           dtype = column_types, sep='\t', na_values='.', encoding="ISO-8859-1", error_bad_lines=False, 
                                           low_memory=False, parse_dates = ["dob", "drop_date", "closed_date", "poffdate"], 
                                           infer_datetime_format = True)
optimized_mb_pbfbxs_ita_0518.columns = optimized_mb_pbfbxs_ita_0518.columns.str.strip().str.lower().str.replace(' ', '_') 
print(mem_usage(optimized_mb_pbfbxs_ita_0518))
optimized_mb_pbfbxs_ita_0518.head()

61.40 MB


,unnamed:_0,campaign_id,hhfilecode,batch,lettercd,ss7,name1,addr2,street,city,state,zip5,zip4,dob,made_unmade,qseqnum,drop_date,closed_date,mla_status,branchnumber,branchcompany,branchstreetaddress,branchcity,branchstate,branchzip,branchphone,branchfax,branchmanager,branchsupervisor,branchstatevp,distance_to_branch,conprofile,bracctno,cifno,mgc,month_split,fname,mname,lname,amt_given1,up_to_offer,from_offer_amount,fico_range_25pt,state1,test_code,fico,poffdate,rmc_phone,rmc_cellphone,unnamed:_48
0,0,FBITA5.0_2018,FB,FMITA,FB,4690645,Christine Pennington,,208 Canvasback Dr.,Bloomingdale,GA,31302,8031,1988-10-23,,1,2018-05-07,2018-06-05,N,1101,Regional Finance,"6409 Abercorn Street, Suite A",Savannah,GA,31405-5796,(912) 354-1144,(912) 328-5408,Debbie Sanders,Mike Votava,Heather Meares,14.808368,,1101003466,B1571550,,,CHRISTINE,,PENNINGTON,NaN,7000,600.0,575-599,GA,,593.0,NaT,(912)547-2071,(900)000-0000,NaN
1,1,FBITA5.0_2018,FB,FMITA,FB,4530510,Tina Rustin,,5 Douglas Ct.,Bloomingdale,GA,31302,9218,1970-11-05,,2,2018-05-07,2018-06-05,N,1101,Regional Finance,"6409 Abercorn Street, Suite A",Savannah,GA,31405-5796,(912) 354-1144,(912) 328-5408,Debbie Sanders,Mike Votava,Heather Meares,13.180202,,1101102475,B1406590,,,TINA,,RUSTIN,NaN,7000,600.0,,GA,,NaN,NaT,(912)856-4204,,NaN
2,2,FBITA5.0_2018,FB,FMITA,FB,8495068,Kenneth Pritsch,,109 Sandstone Dr.,Bloomingdale,GA,31302,8111,1962-07-26,,3,2018-05-07,2018-06-05,N,1101,Regional Finance,"6409 Abercorn Street, Suite A",Savannah,GA,31405-5796,(912) 354-1144,(912) 328-5408,Debbie Sanders,Mike Votava,Heather Meares,18.732519,,1101005213,B1623331,,,KENNETH,A,PRITSCH,NaN,7000,600.0,625-649,GA,,639.0,NaT,(912)330-0626,(218)994-9155,NaN
3,3,FBITA5.0_2018,FB,FMITA,FB,7432608,Angela Parker,,1024 Jones Way SE,Darien,GA,31305,5800,1981-12-02,,4,2018-05-07,2018-06-05,N,1102,Regional Finance,3421-6 Cypress Mill Road,Brunswick,GA,31520-2876,(912) 261-2882,(912) 231-5079,Becky Cross,Mike Votava,Heather Meares,17.899126,,1102002495,B1650604,,,ANGELA,G,PARKER,NaN,7000,600.0,600-624,GA,,608.0,NaT,(912)617-1510,,NaN
4,4,FBITA5.0_2018,FB,FMITA,FB,8354805,Vermiel Pierce Jr.,,1028 Waters Ct. SE,Darien,GA,31305,5828,1975-10-23,,5,2018-05-07,2018-06-05,N,1102,Regional Finance,3421-6 Cypress Mill Road,Brunswick,GA,31520-2876,(912) 261-2882,(912) 231-5079,Becky Cross,Mike Votava,Heather Meares,17.930010,999999999999,1102003613,B1629385,,,VERMIEL,A,PIERCE JR,NaN,7000,600.0,625-649,GA,,626.0,NaT,(912)437-2042,,NaN


In [ ]:
is_ita = read_and_optimized['campaign_id'] == "FBITA5.0_2018"
mb_fb_ita_0518 = read_and_optimized[is_ita]

In [ ]:
mb_fb_ita_0518.shape

In [ ]:
mb_pbfbxs_ita_0518.campaign_id.unique()